### Practice

In [ ]:
# Со сколькими креативными агентствами мы работаем? 
# Креативным агентством считается тот партнер, 
# у которого нет локаций для проведения квестов, 
# но при этом они пишут сценарии, которые мы используем.
SELECT
    COUNT (DISTINCT l.partner_rk) AS uniq_creative_cnt
    --l.partner_rk AS partner_rk_legend,
    --r.partner_rk AS partner_rk_location,
    --r.location_rk AS loc
FROM msu_analytics.legend AS l
LEFT JOIN msu_analytics.location AS r
    ON l.partner_rk = r.partner_rk
WHERE r.partner_rk IS NULL

# >>> 3

In [ ]:
# Выведите название того партнера, на чьих локациях 
# под руководством девушек операторов, 
# среднее время прохождения квеста ниже, 
# чем у всех остальных и укажите это время. 
# В случае неоднозначного ответа, выведите того партнера,
# у которого название лексикографически меньше. 
# (лексикографически – значит меньше по алфавиту)
WITH time_female AS
(
    SELECT
        quest_rk,
        time,
        employee_rk
    FROM msu_analytics.game
    WHERE
        employee_rk IN 
        (
            SELECT
                employee_rk
            FROM
                msu_analytics.employee
            WHERE
                gender_cd = 'f')
),
partner_name AS
(
    SELECT
        c.quest_rk AS quest,
        c.location_rk AS loc,
        c.partner_rk AS partner,
        b.partner_nm AS p_name
    FROM
        (SELECT
            l.quest_rk,
            l.location_rk,
            r.partner_rk
        FROM msu_analytics.quest AS l
            LEFT JOIN msu_analytics.location AS r
            ON l.location_rk = r.location_rk) AS c
        LEFT JOIN msu_analytics.partner AS b
            ON c.partner_rk = b.partner_rk
)

SELECT
    AVG(f.time) AS avg_time,
    p.p_name AS name_partner
    --f.quest_rk, f.time, f.employee_rk,
    --p.quest, p.loc, p.partner, p.p_name
FROM time_female AS f
    LEFT JOIN partner_name AS p
        ON f.quest_rk = p.quest
GROUP BY name_partner
ORDER BY avg_time ASC, name_partner ASC
LIMIT 1

# >>> BMW

In [ ]:
# У какого квеста (выпишите его quest_nm) 
# разница доли состоявшихся квестов в январе и в феврале 
# наибольшая по модулю?
# Долей считать количество состоявшихся квестов поделить на
# количество заявленных. В случае наличия нескольких квестов,
# подходящих под условие, требуется вывести тот,
# у которого значение quest_rk больше.
WITH jan AS
(
    SELECT 
        quest_rk,
        --game_dttm,
        CAST(SUM(game_flg) AS real) / CAST(COUNT(game_flg) AS real) AS success_rate
    FROM msu_analytics.game
    WHERE game_dttm BETWEEN '2018-01-01' AND '2018-02-01'
    GROUP BY quest_rk
),
feb AS 
(
    SELECT 
        quest_rk,
        --game_dttm,
        CAST(SUM(game_flg) AS real) / CAST(COUNT(game_flg) AS real) AS success_rate
    FROM msu_analytics.game
    WHERE game_dttm BETWEEN '2018-02-01' AND '2018-03-01'
    GROUP BY quest_rk
)

SELECT
    j.quest_rk,
    q.quest_nm,
    ABS(j.success_rate - f.success_rate) AS diff
FROM
    jan AS j
    INNER JOIN feb AS f
        ON j.quest_rk = f.quest_rk
    LEFT JOIN msu_analytics.quest as q
        ON j.quest_rk = q.quest_rk
ORDER BY diff DESC, j.quest_rk DESC
LIMIT 1

# >>> "Начало - Москва"